In [ ]:
# -*- coding: utf-8 -*-
# Jupyter cell — LOCO + chr-disjoint HPO with expanded spaces for CatBoost/XGBoost

import os, json, random, warnings
warnings.filterwarnings("ignore")

import optuna
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from functools import partial
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor

# ========== config ==========
SEED = 42
random.seed(SEED); np.random.seed(SEED)

TRAIN_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
VAL_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"

SAVE_ROOT = "../results/hpo_loco_chr_disjoint"
os.makedirs(SAVE_ROOT, exist_ok=True)

META_COLS = ["gene_name","chr","gene_start","gene_end","TSS_start","TSS_end","strand","gex","gex_rank"]
TARGET_R  = "gex_rank"
TARGET_B  = "gex_binary"

# trials per study（依資源調整）
N_TRIALS_LGBM_BIN = 300
N_TRIALS_XGB_BIN  = 300
N_TRIALS_CAT_BIN  = 300
N_TRIALS_CAT_REG  = 300

K_VAL_CHR = 4  # 每個 trial 驗證用的 chr 數

# ========== load ==========
df_x1 = pd.read_csv(TRAIN_PATH, sep="\t"); df_x1["cell_id"] = "X1"
df_x2 = pd.read_csv(VAL_PATH,   sep="\t"); df_x2["cell_id"] = "X2"

df_all = pd.concat([df_x1, df_x2], ignore_index=True)
df_all[TARGET_B] = (df_all["gex"] > 0.0).astype(int)

# features
drop_cols = set(META_COLS + [TARGET_B, "cell_id"])
feature_cols = [c for c in df_all.columns if c not in drop_cols]
chromosomes = sorted(df_all["chr"].unique())
print(f"🧬 chromosomes: {chromosomes}")
print(f"Feature count: {len(feature_cols)} | Total rows: {len(df_all):,}")

# ========== helpers ==========
def split_loco_by_chromosomes(trial, train_cell, val_cell, k_val=K_VAL_CHR):
    """LOCO: train on one cell, validate on the other; chromosomes disjoint."""
    rng = random.Random(SEED)  # 確保每個 trial 不同
    if k_val > len(chromosomes):
        raise ValueError("k_val exceeds available chromosomes.")
    val_chrs = rng.sample(chromosomes, k_val)
    tr = df_all[(df_all["cell_id"] == train_cell) & (~df_all["chr"].isin(val_chrs))].copy()
    va = df_all[(df_all["cell_id"] == val_cell)   & ( df_all["chr"].isin(val_chrs))].copy()
    return tr, va, val_chrs

def mean_per_chr_auc(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        yt = g["_y"].values; yp = g["_p"].values
        if len(np.unique(yt)) == 2 and len(yt) >= 3:
            per.append(roc_auc_score(yt, yp))
    return np.nanmean(per) if per else np.nan

def mean_per_chr_spearman(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        if len(g) >= 3:
            per.append(spearmanr(g["_y"], g["_p"])[0])
    return np.nanmean(per) if per else np.nan
{        loss_function="Logloss",
        eval_metric="AUC",
        iterations=6000,
                random_seed=SEED,
        od_type="Iter", od_wait=200,
        verbose=False,
        bootstrap_type=bootstrap_type,
        'bootstrap_type': 'MVS', 'learning_rate': 0.03210446479623253, 'depth': 7, 'l2_leaf_reg': 1.6920399390073075, 'random_strength': 1.6154397653677859, 'rsm': 0.921748554489924, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 35, 'leaf_estimation_iterations': 9, 'border_count': 186}. Best is trial 218 with value: 0.914366286271263.

def objective_cat_bin(trial, train_cell, val_cell):
    tr, va, _ = split_loco_by_chromosomes(trial, train_cell, val_cell)
    X_tr, y_tr = tr[feature_cols], tr[TARGET_B]
    X_va, y_va = va[feature_cols], va[TARGET_B]

    bootstrap_type = trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
    params = dict(
        loss_function="Logloss",
        eval_metric="AUC",
        iterations=6000,
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.0, 2.0),
        rsm=trial.suggest_float("rsm", 0.5, 1.0),  # feature subsampling
        grow_policy=trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 64),
        leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1, 10),
        border_count=trial.suggest_int("border_count", 128, 255),  # 連續特徵 bin 數
        random_seed=SEED,
        od_type="Iter", od_wait=200,
        verbose=False,
        bootstrap_type=bootstrap_type,
    )
    # 依 bootstrap_type 增加條件參數
    if bootstrap_type == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)
    elif bootstrap_type == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 10.0)
    # MVS 不需要額外參數

    mdl = CatBoostClassifier(**params)
    mdl.fit(X_tr, y_tr, eval_set=(X_va, y_va), verbose=False)
    p = mdl.predict_proba(X_va)[:,1]
    auc_chr = mean_per_chr_auc(va, y_va, p)
    auc_all = roc_auc_score(y_va, p)
    return auc_chr if not np.isnan(auc_chr) else auc_all

def objective_cat_reg(trial, train_cell, val_cell):
    tr, va, _ = split_loco_by_chromosomes(trial, train_cell, val_cell)
    X_tr, y_tr = tr[feature_cols], tr[TARGET_R]
    X_va, y_va = va[feature_cols], va[TARGET_R]

    bootstrap_type = trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
    params = dict(
        loss_function="RMSE",
        eval_metric="RMSE",
        iterations=6000,
        learning_rate=trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.0, 2.0),
        rsm=trial.suggest_float("rsm", 0.5, 1.0),
        grow_policy=trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 64),
        leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1, 10),
        border_count=trial.suggest_int("border_count", 128, 255),
        random_seed=SEED,
        od_type="Iter", od_wait=200,
        verbose=False,
        bootstrap_type=bootstrap_type,
    )
    if bootstrap_type == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)
    elif bootstrap_type == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 10.0)

    mdl = CatBoostRegressor(**params)
    mdl.fit(X_tr, y_tr, eval_set=(X_va, y_va), verbose=False)
    pred = mdl.predict(X_va)
    rho_chr = mean_per_chr_spearman(va, y_va, pred)
    rho_all = spearmanr(y_va, pred)[0]
    return rho_chr if not np.isnan(rho_chr) else rho_all

# ========== run studies for both LOCO directions ==========
def run_and_save(name, objective_fn, n_trials, train_cell, val_cell):
    tag = f"{name}_{train_cell}_to_{val_cell}"
    print(f"\n===== 🔎 Start study: {tag} | trials={n_trials} =====")
    study = optuna.create_study(direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(partial(objective_fn, train_cell=train_cell, val_cell=val_cell),
                   n_trials=n_trials, show_progress_bar=True)
    print(f"✅ {tag} Best value: {study.best_value}")
    print("🏆 Best params:")
    for k,v in study.best_params.items():
        print(f"  {k}: {v}")
    out_dir = os.path.join(SAVE_ROOT, tag); os.makedirs(out_dir, exist_ok=True)
    study.trials_dataframe().to_csv(os.path.join(out_dir, "trials_log.csv"), index=False)
    with open(os.path.join(out_dir, "best_params.json"), "w") as f:
        json.dump(study.best_params, f, indent=4)
    print(f"💾 Saved to {out_dir}")
    return study

PAIRS = [("X1","X2"), ("X2","X1")]

for a,b in PAIRS:
    run_and_save("catboost_bin", objective_cat_bin,  N_TRIALS_CAT_BIN,  a, b)
    run_and_save("catboost_reg", objective_cat_reg,  N_TRIALS_CAT_REG,  a, b)


[I 2025-10-23 01:53:31,126] A new study created in memory with name: no-name-1eba3d9a-944c-44c3-a5c7-7932257a4125


🧬 chromosomes: ['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']
Feature count: 2892 | Total rows: 32,568

===== 🔎 Start study: catboost_bin_X1_to_X2 | trials=300 =====


Best trial: 0. Best value: 0.907955:   0%|          | 1/300 [00:37<3:07:28, 37.62s/it]

[I 2025-10-23 01:54:08,742] Trial 0 finished with value: 0.9079549137525209 and parameters: {'bootstrap_type': 'Bernoulli', 'learning_rate': 0.060099747183803134, 'depth': 5, 'l2_leaf_reg': 2.403950683025824, 'random_strength': 0.11616722433639892, 'rsm': 0.9330880728874675, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 63, 'leaf_estimation_iterations': 9, 'border_count': 155, 'subsample': 0.5909124836035503}. Best is trial 0 with value: 0.9079549137525209.


Best trial: 1. Best value: 0.909475:   1%|          | 2/300 [01:23<3:32:20, 42.75s/it]

...

[I 2025-10-23 17:53:55,320] Trial 243 finished with value: 0.9103100845753387 and parameters: {'bootstrap_type': 'MVS', 'learning_rate': 0.026805906603488264, 'depth': 7, 'l2_leaf_reg': 1.7278606394563103, 'random_strength': 1.6022374691215022, 'rsm': 0.8990224996483708, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 44, 'leaf_estimation_iterations': 10, 'border_count': 182}. Best is trial 218 with value: 0.914366286271263.


Best trial: 218. Best value: 0.914366:  82%|████████▏ | 245/300 [16:02:31<1:36:54, 105.73s/it]

[I 2025-10-23 17:56:02,421] Trial 244 finished with value: 0.9122804722707132 and parameters: {'bootstrap_type': 'MVS', 'learning_rate': 0.01831455229403545, 'depth': 7, 'l2_leaf_reg': 2.160877863188706, 'random_strength': 0.8514867148878529, 'rsm': 0.8894921936753288, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 43, 'leaf_estimation_iterations': 10, 'border_count': 189}. Best is trial 218 with value: 0.914366286271263.


Best trial: 218. Best value: 0.914366:  82%|████████▏ | 246/300 [16:10:31<3:16:21, 218.17s/it]

[I 2025-10-23 18:04:02,966] Trial 245 finished with value: 0.911818946025106 and parameters: {'bootstrap_type': 'MVS', 'learning_rate': 0.01637892806790006, 'depth': 7, 'l2_leaf_reg': 2.034750739411999, 'random_strength': 1.6476596651061257, 'rsm': 0.9057431437959109, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 45, 'leaf_estimation_iterations': 10, 'border_count': 186}. Best is trial 218 with value: 0.914366286271263.


In [1]:
# -*- coding: utf-8 -*-
# Jupyter cell — LOCO + chr-disjoint HPO with expanded spaces for CatBoost/XGBoost

import os, json, random, warnings
warnings.filterwarnings("ignore")

import optuna
import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from functools import partial
from sklearn.metrics import roc_auc_score

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier, CatBoostRegressor

# ========== config ==========
SEED = 42
random.seed(SEED); np.random.seed(SEED)

TRAIN_PATH = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X1_all_rank_features_with_y.tsv"
VAL_PATH   = "../preprocessed_data/reference/1. merged data/with_y_100_one_side/X2_all_rank_features_with_y.tsv"

SAVE_ROOT = "../results/hpo_loco_chr_disjoint"
os.makedirs(SAVE_ROOT, exist_ok=True)

META_COLS = ["gene_name","chr","gene_start","gene_end","TSS_start","TSS_end","strand","gex","gex_rank"]
TARGET_R  = "gex_rank"
TARGET_B  = "gex_binary"

# trials per study（依資源調整）
N_TRIALS_LGBM_BIN = 300
N_TRIALS_XGB_BIN  = 300
N_TRIALS_CAT_BIN  = 300
N_TRIALS_CAT_REG  = 300

K_VAL_CHR = 4  # 每個 trial 驗證用的 chr 數

# ========== load ==========
df_x1 = pd.read_csv(TRAIN_PATH, sep="\t"); df_x1["cell_id"] = "X1"
df_x2 = pd.read_csv(VAL_PATH,   sep="\t"); df_x2["cell_id"] = "X2"

df_all = pd.concat([df_x1, df_x2], ignore_index=True)
df_all[TARGET_B] = (df_all["gex"] > 0.0).astype(int)

# features
drop_cols = set(META_COLS + [TARGET_B, "cell_id"])
feature_cols = [c for c in df_all.columns if c not in drop_cols]
chromosomes = sorted(df_all["chr"].unique())
print(f"🧬 chromosomes: {chromosomes}")
print(f"Feature count: {len(feature_cols)} | Total rows: {len(df_all):,}")

# ========== helpers ==========
def split_loco_by_chromosomes(trial, train_cell, val_cell, k_val=K_VAL_CHR):
    """LOCO: train on one cell, validate on the other; chromosomes disjoint."""
    rng = random.Random(SEED)  # 確保每個 trial 不同
    if k_val > len(chromosomes):
        raise ValueError("k_val exceeds available chromosomes.")
    val_chrs = rng.sample(chromosomes, k_val)
    tr = df_all[(df_all["cell_id"] == train_cell) & (~df_all["chr"].isin(val_chrs))].copy()
    va = df_all[(df_all["cell_id"] == val_cell)   & ( df_all["chr"].isin(val_chrs))].copy()
    return tr, va, val_chrs

def mean_per_chr_auc(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        yt = g["_y"].values; yp = g["_p"].values
        if len(np.unique(yt)) == 2 and len(yt) >= 3:
            per.append(roc_auc_score(yt, yp))
    return np.nanmean(per) if per else np.nan

def mean_per_chr_spearman(df_v, y_true, y_pred):
    per = []
    tmp = df_v.assign(_y=np.asarray(y_true), _p=np.asarray(y_pred))
    for _, g in tmp.groupby("chr"):
        if len(g) >= 3:
            per.append(spearmanr(g["_y"], g["_p"])[0])
    return np.nanmean(per) if per else np.nan

         {loss_function="RMSE",
        eval_metric="RMSE",
        iterations=6000,
                random_seed=SEED,
        od_type="Iter", od_wait=200,
        verbose=False,
        bootstrap_type=bootstrap_type,
      'bootstrap_type': 'Bernoulli', 'learning_rate': 0.023746198818402044, 'depth': 7, 'l2_leaf_reg': 2.6636900997297435, 'random_strength': 1.9391692555291171, 'rsm': 0.8875664116805573, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 59, 'leaf_estimation_iterations': 1, 'border_count': 153, 'subsample': 0.522613644455269}. Best is trial 3 with value: 0.785300664439774.
 
def objective_cat_reg(trial, train_cell, val_cell):
    tr, va, _ = split_loco_by_chromosomes(trial, train_cell, val_cell)
    X_tr, y_tr = tr[feature_cols], tr[TARGET_R]
    X_va, y_va = va[feature_cols], va[TARGET_R]

    bootstrap_type = trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
    params = dict(
        loss_function="RMSE",
        eval_metric="RMSE",
        iterations=6000,
        learning_rate=trial.suggest_float("learning_rate", 0.005, 0.1, log=True),
        depth=trial.suggest_int("depth", 4, 10),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        random_strength=trial.suggest_float("random_strength", 0.0, 2.0),
        rsm=trial.suggest_float("rsm", 0.5, 1.0),
        grow_policy=trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        min_data_in_leaf=trial.suggest_int("min_data_in_leaf", 1, 64),
        leaf_estimation_iterations=trial.suggest_int("leaf_estimation_iterations", 1, 10),
        border_count=trial.suggest_int("border_count", 128, 255),
        random_seed=SEED,
        od_type="Iter", od_wait=200,
        verbose=False,
        bootstrap_type=bootstrap_type,
    )
    if bootstrap_type == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample", 0.5, 1.0)
    elif bootstrap_type == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0.0, 10.0)

    mdl = CatBoostRegressor(**params)
    mdl.fit(X_tr, y_tr, eval_set=(X_va, y_va), verbose=False)
    pred = mdl.predict(X_va)
    rho_chr = mean_per_chr_spearman(va, y_va, pred)
    rho_all = spearmanr(y_va, pred)[0]
    return rho_chr if not np.isnan(rho_chr) else rho_all

# ========== run studies for both LOCO directions ==========
def run_and_save(name, objective_fn, n_trials, train_cell, val_cell):
    tag = f"{name}_{train_cell}_to_{val_cell}"
    print(f"\n===== 🔎 Start study: {tag} | trials={n_trials} =====")
    study = optuna.create_study(direction="maximize",
                                sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(partial(objective_fn, train_cell=train_cell, val_cell=val_cell),
                   n_trials=n_trials, show_progress_bar=True)
    print(f"✅ {tag} Best value: {study.best_value}")
    print("🏆 Best params:")
    for k,v in study.best_params.items():
        print(f"  {k}: {v}")
    out_dir = os.path.join(SAVE_ROOT, tag); os.makedirs(out_dir, exist_ok=True)
    study.trials_dataframe().to_csv(os.path.join(out_dir, "trials_log.csv"), index=False)
    with open(os.path.join(out_dir, "best_params.json"), "w") as f:
        json.dump(study.best_params, f, indent=4)
    print(f"💾 Saved to {out_dir}")
    return study

PAIRS = [("X1","X2"), ("X2","X1")]

for a,b in PAIRS:
    run_and_save("catboost_reg", objective_cat_reg,  N_TRIALS_CAT_REG,  a, b)


[I 2025-10-23 18:13:56,976] A new study created in memory with name: no-name-e3ca706c-dd73-4799-bd62-964389964128


🧬 chromosomes: ['chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr2', 'chr20', 'chr21', 'chr22', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9']
Feature count: 2892 | Total rows: 32,568

===== 🔎 Start study: catboost_reg_X1_to_X2 | trials=300 =====


Best trial: 0. Best value: 0.784287:   0%|          | 1/300 [1:36:45<482:09:36, 5805.27s/it]

[I 2025-10-23 19:50:42,269] Trial 0 finished with value: 0.7842867620946721 and parameters: {'bootstrap_type': 'Bernoulli', 'learning_rate': 0.030049873591901578, 'depth': 5, 'l2_leaf_reg': 2.403950683025824, 'random_strength': 0.11616722433639892, 'rsm': 0.9330880728874675, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 63, 'leaf_estimation_iterations': 9, 'border_count': 155, 'subsample': 0.5909124836035503}. Best is trial 0 with value: 0.7842867620946721.


Best trial: 0. Best value: 0.784287:   1%|          | 2/300 [1:38:45<203:43:23, 2461.08s/it]

...

Best trial: 31. Best value: 0.789598:  17%|█▋        | 52/300 [4:55:01<10:51:39, 157.66s/it]

[I 2025-10-23 23:08:58,507] Trial 51 finished with value: 0.7875221169807626 and parameters: {'bootstrap_type': 'Bernoulli', 'learning_rate': 0.011574646428123636, 'depth': 7, 'l2_leaf_reg': 5.383830688991296, 'random_strength': 0.14688771916708587, 'rsm': 0.812548796884496, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 20, 'leaf_estimation_iterations': 7, 'border_count': 213, 'subsample': 0.685907655084214}. Best is trial 31 with value: 0.7895976862259292.


Best trial: 31. Best value: 0.789598:  17%|█▋        | 52/300 [4:58:34<23:43:59, 344.51s/it]


[W 2025-10-23 23:12:31,624] Trial 52 failed with parameters: {'bootstrap_type': 'Bernoulli', 'learning_rate': 0.009513000995566671, 'depth': 8, 'l2_leaf_reg': 4.581673487697778, 'random_strength': 0.435917201021137, 'rsm': 0.7411991368842427, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 1, 'border_count': 219, 'subsample': 0.6318627436774725} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/7g/j5s0yzcj34l3v043s7znkplc0000gn/T/ipykernel_19080/4286109250.py", line 113, in objective_cat_reg
    mdl.fit(X_tr, y_tr, eval_set=(X_va, y_va), verbose=False)
  File "/opt/anaconda3/envs/ml4g_project1/lib/python3.12/site-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, te

KeyboardInterrupt: 